# Captura y almacenamiento de la información
Profesores: MSc Oscar Bría - Mg Javier Bazzocco


## Introducción y objetivos

Twitter es una de las mayores redes sociales en cuanto al flujo de información. Dada su gran popularidad entre personas de distintas ubicaciones geográficas, edades, aspectos sociales, etc...en esta red puede encontrarse información de las mas diversas areas (politica, deportes, ciencia, chismes, etc...). Si bien un tweet puede tener un número bajo de caracteres, la información suele estar bien resumida y esto lo hace una herramienta ágil para la búsqueda inmediata de información sobre algún tema particular. Por ejemplo, basta buscar con un hashtag (#palabraclave) sobre un tema de interés para que tengamos la información mas actualizada sobre esa palabra clave. Hoy en día no solo las personas utilizan estas redes sociales para transmitir información, sino que también lo hacen las empresas, instituciones educativas y científicas, gobiernos, etc...



En este contexto, el objetivo de esta notebook es analizar si hay alguna respuesta del público de Twitter a un evento natural como un terremoto. Claramente esto va a depender de la ubicación del evento sísmico y las personas cercanas a este punto que posteen algo en sus cuentas de este red social. La información de los eventos sísmicos se obtiene a partir de una base de datos siendo de interés las coordenas geográficas y magnitud del sismo.

### Comenzemos...

Para este análisis se utilizará el lenguaje de programación Python y la librería Tweepy para interactuar con la API de Tweeter. También se utilizarán otras como Pandas y csv para el manejo de los datos. Los datos serán obtenidos de alguna institución que permite el acceso libre a los datos y serán almacenados en MongoDB. Utilizo MongoDB Atlas, esto es la base de datos como servicio de MongoDB (DaS) el cual me permite tener una cuenta gratuita con un espacio limitado pero suficiente para este trabajo. https://www.mongodb.com/cloud/atlas/signup?v=1

In [11]:
# Se importan las librerías 
import tweepy as tw # para interactuar con la API de Twitter
import pandas as pd # para manipular datos
import csv          # archivos comma separation value
import json         # archivos JSON
#!pip install dnspython # para instalar librerías para poder concetar con MongoDB 
#!pip install pymongo   # permite interacción entre MongoDB y Python  

from pymongo import MongoClient

client = MongoClient("")# completar con sus datos.

In [12]:
db = client.get_database('Prueba0') # Me meto en la base de datos Prueba0 que generé en mi cluster en MongoDB Atlas.
db

Database(MongoClient(host=['cluster0-shard-00-01-rkxjw.mongodb.net:27017', 'cluster0-shard-00-02-rkxjw.mongodb.net:27017', 'cluster0-shard-00-00-rkxjw.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='Cluster0-shard-0', ssl=True), 'Prueba0')

In [13]:
# Cargo el archivo JSON con la información de los terremotos.

collection_sismos = db['Sismos'] # coleccion donde quiero guardar el documento

with open('query_usgs.json') as f:
    file_data = json.load(f)

# Para insertar UN documento:
#collection_sismos.insert_one(file_data) #Obs: Si corro esta celda carga el dato en la colección en mi base de datos cluster0
# Para insertar VARIOS documentos:
#collection_sismos.insert_many(file_data)

client.close()

In [14]:
# me canse de probar cosas...no funciona. :(
# build a Python dictionary for query
#query = {"mag" : 4.9}

#documents = list(collection_sismos.find(query))
#for doc in documents:
#    print ("\ndoc _id:", doc["_id"])

Ahora bien...ya tenemos la información de los terremotos desde Mongo. Para poder acceder a la información de Twitter hay que tener una cuenta en twitter developers (https://developer.twitter.com/en). Ahí se crea una aplicación luego de un par de preguntas sobre el objetivo de la App y listo! Se necesitan 4 cosas de la aplicacion de twitter.

In [15]:
consumer_key= 'sus datos'
consumer_secret= 'sus datos'
access_token= 'sus datos'
access_token_secret= 'sus datos'

Para crear la conexión...

In [16]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

Una vez que está todo preparado para interactuar con Twitter podemos comenzar a trabajar para lograr nuestro objetivo. Debe de considerarse que nuestro planeta está en constante movimiento y por ende se registran muchísimos terremotos por día. No obstante, son los de magintudes grandes y algunos medianos, con un escenario geológico particular, los que generalmente se pueden percibir. Cuando este es el caso, se ha visto y es de esperarse, las personas suelen postear en sus cuentas de redes sociales sobre como y/o cuando percibieron el movimiento.

Para ilustrar esto se puede ver facilmente los últimos tweets del INSTITUTO NACIONAL DE PREVENCIÓN SÍSMICA ( www.inpres.gob.ar ). Es inmediato que las magnitudes registradas por los sismógrafos, en general, no son perceptibles por los humanos. Puede haber excepciones! Por otro lado, la mayoría de las instituciones y/o cuentas que comparten este tipo de información no suelen dar la ubicación geográfica del evento. En el mejor de los casos tendremos esta información de ubicación en los niveles de provincia o estado. Tal vez con suerte, de localidad o ciudad. 



In [17]:
# usuario de twitter
name = 'inpres_sismos'
#name = 'USGSBigQuakes'
nro_tweets = 10 # cantidad de tweets que queremos ver

resultado = api.user_timeline(id=name, count=nro_tweets) # interacción con Twitter

for tweet in resultado: 
    print(tweet.text)

Se ha detectado un #sismo de 2.5 ML en La Rioja. 

Más info en el enalce. 
https://t.co/rOXRfppMII
Se ha detectado un #sismo de 2.8 ML en Cordoba. 

Más info en el enalce. 
https://t.co/zK7ih4tgLq
Se ha detectado un #sismo de 2.6 ML en San Juan. 

Más info en el enalce. 
https://t.co/r8yiZwFNGW
Se ha detectado un #sismo de 2.8 ML en Jujuy. 

Más info en el enalce. 
https://t.co/lZXt9KpQPi
Se ha detectado un #sismo de 2.6 ML en La Rioja. 

Más info en el enalce. 
https://t.co/I94EZZ42OY
Se ha detectado un #sismo de 2.9 ML en Salta. 

Más info en el enalce. 
https://t.co/jSNPDRyQfP
Se ha detectado un #sismo de 2.8 ML en San Juan. 

Más info en el enalce. 
https://t.co/u4vnLZFksu
Se ha detectado un #sismo de 2.5 ML en San Juan. 

Más info en el enalce. 
https://t.co/vhI4tb2esB
Se ha detectado un #sismo de 2.9 ML en Salta. 

Más info en el enalce. 
https://t.co/SQH7ICvKZs
Se ha detectado un #sismo de 3.2 ML en Salta. 

Más info en el enalce. 
https://t.co/VswMvm9JXd


Es inmediato que la mayoría de las magnitudes registradas por los sismógrafos no son perceptibles por los humanos. Por otro lado, la mayoría de las instituciones y/o cuentas que comparten este tipo de información no suelen dar la ubicación geográfica del evento. En el mejor de los casos tendremos esta información de ubicación en los niveles de provincia o estado. Tal vez con suerte, de la localidad o ciudad. Por esto, para ver si es posible o no ver la reacción de las personas a un evento de este tipo, es necesario contar con los datos de alguna institución. En el siguiente link puede encontrarse el catalogo del USGS https://earthquake.usgs.gov/earthquakes/search/ , el cual permite bajar los datos en diferentes formatos y son los que fueron cargados en la BD anteriormente. 

Como mencioné, probablemente se pueda encontrar los tweets relacionados con algún terremoto de magnitud alta. Para esto buscamos en la base de datos por el terremoto de mayor magnitud y nos interesa sus coordenadas geográficas para buscar en Twitter.


### Acá iría el query para buscar el terremoto de mayor magnitud y guardar en variables la latitud y longitud

Se debe de tener en cuenta de que una coordenada geográfica es un punto en nuestro planeta por lo que si queremos buscar tweets, debemos buscar mas que en el epicentro del evento sísmico en los alrededores. La API de Tweeter mediante Tweepy nos permite elegir el radio entorno a este punto donde queremos ver los tweets. 

In [18]:
#Esta info no va en la version final...es para probar que funcione.
# posición de la ubicación de interés. En este caso Buenos Aires y San Juan, Argentina:

latitud = -34.6131516#    -31.5375004#
longitud = -58.3772316#   -68.5363922 

In [19]:
radio = 700             # Rango de búsqueda en km en torno a la posición dada.
max_res = 10            # numero maximo de resultados que quiero ver.
outfile = "output.csv"  # por si quiero guardar los datos en un archivo tipo csv

Algo importante para este análisis es la elección de la palabra clave para buscar los tweets. Tener en cuenta que se pueden obtener los tweets de hasta los últimos 7 días.

In [21]:
search_words = "#terremoto"+ "-filter:retweets" 
desde = "2020-4-18" # la fecha (ano/mes/dia) desde...
hasta =  "2020-4-26" # hasta...

# Extraemos los tweets...
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="es",
              geocode = "%f,%f,%dkm" % (latitud, longitud, radio),
              since=desde,
              until=hasta).items(max_res)
tweets

#[tweet.text for tweet in tweets]
users_locs = [[tweet.user.screen_name, tweet.user.location, tweet.text] for tweet in tweets]
users_locs

tweet_text = pd.DataFrame(data=users_locs, 
                    columns=['user', "location","Tweets"])
tweet_text#.to_csv('out.csv',index=False, sep='\t')

,user,location,Tweets
0,mar2san,Uruguay,"Va a venir la NASA,en serio República oriental..."
1,LaSolapaOk,"Entre Ríos, Argentina",#Entrerrianos | #ColoColo 🎉🎈\n\nHoy el conjunt...
